In [54]:
import pandas as pd

In [62]:
def room_division(dataRoom):
    """
    room_division
    -------------
    take all the data and divide the rooms of all PoliTo
    ### Output:
        - the dataframe with the room divided
    """
    Ap = "AP-AULA"
    dataRoom = dataRoom[dataRoom["name_ap"].notnull()]
    dataRoom = dataRoom[dataRoom["name_ap"].str.contains(Ap)]
    rooms = pd.DataFrame()
    rooms[["AP", "Room", "APnum", "NaN"]] = dataRoom["name_ap"].str.split(
        "-", expand=True
    )
    dataRoom = pd.concat([dataRoom, rooms], axis=1)
    dataRoom = dataRoom.drop(["AP", "NaN"], axis=1)
    return dataRoom

In [101]:
df = pd.read_csv("ml_data/03_08_2024_17_40_56_AULA3P_120.csv")
df.drop(columns=["Unnamed: 0", "Timestamp_y", "channel_1", "channel_2", "class", "domain", "client_type", "code_ap"], inplace=True)
df = room_division(df)

In [102]:

df_aula = df[df["Room"] == "AULA10"]

In [116]:
df_aula

,Timestamp_x,MAC_masked,user_masked,rssi,snr,byte_rx,byte_tx,name_ap,ch_utilization_1,ch_utilization_2,noise_x,noise_y,tx_power_1,tx_power_2,n_clients_2_4,n_clients_5,Room,APnum
502,1.709920e+09,bf6afba33f7a046e0659934491bca4102942456570fc5e...,d3112e186ea08acec5cc8778160ad7e718f0be45709bd9...,-64,33,4.365169e+07,16069930.0,AP-AULA10,22.0,0.0,-98.0,-94.0,1.0,1.0,2.0,7.0,AULA10,None
595,1.709920e+09,47458a2bf9f227b31a3ba95b3d579369898b52d449c9f3...,6039f7c7802344c3a242e0025e446395538421f6e5ca5b...,-77,18,2.475676e+07,2013528.0,AP-AULA10,22.0,0.0,-98.0,-94.0,1.0,1.0,2.0,7.0,AULA10,None
652,1.709920e+09,1953b63ceb18082370d646b872e287b7feaf34d397fc6a...,98cb3e55eeb9f2c56c14a8fad3f5ee1eb7cb6343e5465b...,-82,13,2.783360e+09,46230296.0,AP-AULA10,22.0,0.0,-98.0,-94.0,1.0,1.0,2.0,7.0,AULA10,None
685,1.709920e+09,fc67f514dcb4258c26770ea6b80b7fb3f3ce5e25ac45f3...,3529214424a5c112145a366e28a02d1261b8914d91ef5f...,-80,15,3.473733e+08,6680646.0,AP-AULA10,22.0,0.0,-98.0,-94.0,1.0,1.0,2.0,7.0,AULA10,None
996,1.709920e+09,63f7d92ed64f57b11f73bc84ee67ad08f686f0b742a209...,88a19f18862439c53f9b444a831f116582da19fba35ad5...,-69,26,3.159775e+08,7178542.0,AP-AULA10,22.0,0.0,-98.0,-94.0,1.0,1.0,2.0,7.0,AULA10,None
1842,1.709920e+09,798a89338e3ca4ccc04a65841c39537c826d0f9f1495b0...,1a0b7989f13cb7afe0894b27fdbe084555c5e6ef7b4044...,-79,16,2.462623e+06,1263717.0,AP-AULA10,22.0,0.0,-98.0,-94.0,1.0,1.0,2.0,7.0,AULA10,None


In [117]:
n_devices = df_aula["n_clients_2_4"].iloc[0] + df_aula["n_clients_5"].iloc[0]
df_dist_users = df_aula.drop_duplicates(subset="user_masked")
n_users = len(df_dist_users[df_dist_users["snr"] > 20])

9.0